In [1]:
!pip install datasets==2.14.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.6.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [2]:
!pip install transformers datasets torch scikit-learn streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 117.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable


 # Load and preprocess data

In [5]:
import shutil

# Delete old/broken cache
shutil.rmtree('/root/.cache/huggingface/datasets', ignore_errors=True)

# Then load the dataset again
from datasets import load_dataset
ds = load_dataset("ag_news")

print(ds)


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


# Tokenize and Rrain BERT!

In [6]:
from transformers import BertTokenizer

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the dataset
def tokenize_function(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=128)

tokenized_ds = ds.map(tokenize_function, batched=True)

# Rename label column to 'labels' as required by Trainer
tokenized_ds = tokenized_ds.rename_column('label', 'labels')

# Set format for PyTorch
tokenized_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

print(tokenized_ds)


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7600
    })
})


# Fine-tune BERT

In [7]:
!pip install transformers datasets scikit-learn


In [8]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments


In [9]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
!pip install --upgrade transformers


# Define training arguments

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    save_strategy="epoch",   # ✅ very important: save model every epoch
    report_to="none"         # ✅ disables wandb etc.
)


In [1]:
!pip install numpy==1.26.4


# Create Trainer and train

In [12]:
from transformers import BertForSequenceClassification, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {"accuracy": acc, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    compute_metrics=compute_metrics
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.404100
1000,0.289800
1500,0.269800
2000,0.258200
2500,0.264100
3000,0.256200
3500,0.227600
4000,0.232200
4500,0.222500
5000,0.218200


Step,Training Loss
500,0.404100
1000,0.289800
1500,0.269800
2000,0.258200
2500,0.264100
3000,0.256200
3500,0.227600
4000,0.232200
4500,0.222500
5000,0.218200


TrainOutput(global_step=15000, training_loss=0.190507527923584, metrics={'train_runtime': 5120.4261, 'train_samples_per_second': 46.871, 'train_steps_per_second': 2.929, 'total_flos': 1.578694680576e+16, 'train_loss': 0.190507527923584, 'epoch': 2.0})

In [ ]:
# Evaluate on the test set
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 0.18572735786437988, 'eval_accuracy': 0.9513157894736842, 'eval_f1': 0.9513174750428515, 'eval_runtime': 51.0817, 'eval_samples_per_second': 148.781, 'eval_steps_per_second': 9.299, 'epoch': 2.0}


In [19]:
!pip install gradio


In [28]:
import gradio as gr
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# Load tokenizer and trained model (adjust checkpoint path if needed)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('./results/checkpoint-15000')

labels = ["World", "Sports", "Business", "Sci/Tech"]

def classify_news(headline):
    inputs = tokenizer(headline, return_tensors='pt', padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        logits = model(**inputs).logits
    pred = logits.argmax().item()
    return f"📰 Predicted Topic: **{labels[pred]}**"

# CSS to set newspaper background and style the textbox and output
custom_css = """
.gradio-container {
    background: url('https://images.unsplash.com/photo-1522202176988-66273c2fd55f?ixlib=rb-4.0.3&auto=format&fit=crop&w=1950&q=80');
    background-size: cover;
    background-attachment: fixed;
    display: flex;
    flex-direction: column;
    align-items: center;
    justify-content: flex-start;    /* align items from the top */
    padding-top: 120px;             /* push everything down */
}
h1, p {
     color: #ffffff !important;
    text-shadow: none;
    text-align: center;
}
textarea {
    background-color: rgba(255, 255, 255, 0.8) !important;
    color: #000000 !important;
    margin: 0 auto;
}
.output-markdown {
    color: #ffffff !important;
    text-align: center !important;
    font-size: 20px !important;
    text-shadow: none;
    margin-top: 20px;
}

"""



# Build Gradio interface
demo = gr.Interface(
    fn=classify_news,
    inputs=gr.Textbox(lines=2, placeholder="Enter a news headline..."),
    outputs=gr.Markdown(),
    title="📰 News Topic Classifier",
    description="Enter a news headline",
    theme="gradio/soft",  # You can choose other themes too, e.g., gradio/gstaff, gradio/peach
    css=custom_css
)

# Launch with public link
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1298ff9dcd4025ba8a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
